In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from tensorflow.keras import  callbacks
from tensorflow.keras.optimizers import Adam

In [5]:
def engineer_features(df):
    """Add engineered features to better capture sales patterns"""
    df_enhanced = df.copy()
    
    # Add rolling statistics
    df_enhanced['rolling_mean_7d'] = df_enhanced['Units Sold'].rolling(window=7).mean()
    df_enhanced['rolling_std_7d'] = df_enhanced['Units Sold'].rolling(window=7).std()
    
    # Add cyclical time features
    df_enhanced['day_sin'] = np.sin(2 * np.pi * df_enhanced['Day']/31.0)
    df_enhanced['day_cos'] = np.cos(2 * np.pi * df_enhanced['Day']/31.0)
    df_enhanced['month_sin'] = np.sin(2 * np.pi * df_enhanced['Month']/12.0)
    df_enhanced['month_cos'] = np.cos(2 * np.pi * df_enhanced['Month']/12.0)
    
    # Add lag features
    df_enhanced['lag_1'] = df_enhanced['Units Sold'].shift(1)
    df_enhanced['lag_7'] = df_enhanced['Units Sold'].shift(7)
    
    # Fill NaN values created by rolling windows and lags
    df_enhanced = df_enhanced.fillna(method='bfill')
    
    return df_enhanced

In [6]:
def generate_sequences(df, sequence_length = 60):
    X, y  = [], []
    for i in range(len(df)- sequence_length):
        j = i+sequence_length
        X.append(df.iloc[i : j].values)
        y.append(df["Units Sold"].iloc[j])

    X = np.array(X)
    y = np.array(y)
    return X, y

In [7]:
def get_model(X):
    lstm_model = keras.Sequential([
        layers.Input(shape=(X.shape[1], X.shape[2])),
        layers.BatchNormalization(),

        layers.Bidirectional(layers.LSTM(128, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.01))),
        layers.BatchNormalization(),
        layers.Dropout(0.2),

        layers.Bidirectional(layers.LSTM(64, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.01))),
        layers.BatchNormalization(),
        layers.Dropout(0.2),


        layers.Bidirectional(layers.LSTM(32)),
        layers.BatchNormalization(),
        layers.Dropout(0.2),


        layers.Dense(32, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(16, activation='relu'),
        layers.Dense(1),
    ])


    optimizer = tf.keras.optimizers.AdamW(
        learning_rate=0.001,
        weight_decay=0.001
    )

    lstm_model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.Huber(delta=1.0),
        metrics=['mae', 'mse']
    )
    return lstm_model

In [10]:
def get_callbacks(patience_lr=5, patience_stop=10):
    callbacks_list = [
        # Learning rate reduction
        callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=patience_lr,
            min_lr=1e-6,
            verbose=1
        ),
        # Early stopping
        callbacks.EarlyStopping(
            monitor='val_loss',
            patience=patience_stop,
            restore_best_weights=True,
            verbose=1
        ),
    ]
    return callbacks_list

In [11]:
def train_model(X_train, y_train, model, callbacks_list, epochs=50):
    batch_size = min(64, 2**int(np.log2(len(X_train)/100)))
    callbacks_list = get_callbacks()

    history = model.fit(
        X_train, 
        y_train, 
        epochs=epochs, 
        batch_size=batch_size, 
        verbose=1, 
        validation_split=0.2, 
        callbacks=callbacks_list
    )
    return history

In [12]:
def plot_loss(history):
    train_loss = history.history['loss']
    val_loss = history.history['val_loss'] 
    plt.figure(figsize=(8, 5))  # Adjust figure size as needed# Label accordingly
    plt.title('Training and Validation Loss/Accuracy')
    plt.plot(train_loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend()
    plt.show()